# [Ateliers: Technologies de l'intelligence Artificielle](https://github.com/wikistat/AI-Frameworks)

<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 
<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" width=400, style="max-width: 150px; display: inline"  alt="Wikistat"/></a>
<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" width=400,  style="float:right;  display: inline" alt="IMT"/> </a>
    
</center>

# LAB 4 Introduction to Reinforcement Learning  -  Part 2 : Q-Learning & Deep Q-Network
The objectives of this noteboks are the following : 

* Implement Q-Learning on simple Markov Decision Process
* Implement DQN on a pacman-like game (gridworld)
* Implement D3QN on the same game
 
Source : [https://github.com/ageron/handson-ml](https://github.com/ageron/handson-ml) and https://github.com/breeko/Simple-Reinforcement-Learning-with-Tensorflow/blob/master/Part%202%20-%20Policy-based%20Agents%20with%20Keras.ipynb

## Import librairies

In [ ]:
import numpy as np
import random
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot figures and animations
%matplotlib inline
%matplotlib nbagg
import matplotlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from IPython.display import HTML


plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12


import tensorflow.keras.models as km
import tensorflow.keras.layers as kl
import tensorflow.keras.initializers as ki
import tensorflow.keras.optimizers as ko
import tensorflow.keras.losses as klo
import tensorflow.keras.backend as K


# Gym Librairy
import gym
import pandas as pd


def update_scene(num, frames, patch):
    patch.set_data(frames[num])
    return patch,

def plot_animation(frames, repeat=False, interval=400):
    plt.close()  # or else nbagg sometimes plots in the previous cell
    fig = plt.figure()
    patch = plt.imshow(frames[0])
    plt.axis('off')
    return animation.FuncAnimation(fig, update_scene, fargs=(frames, patch), frames=len(frames), repeat=repeat, interval=interval)

## Q-Learning on Markov Decision Process

### Definition

We will first define a simple markov process on wich we will apply Q-learning algorithm.

here is an illustration of the MDP that we will define.

![images](images/mdp.png)


We store the different action and transition probabilities within a list

In [ ]:
transition_probabilities = [
        [[0.7, 0.3, 0.0], [1.0, 0.0, 0.0], [0.8, 0.2, 0.0]], 
        [[0.0, 1.0, 0.0], None, [0.0, 0.0, 1.0]],
        [None, [0.8, 0.1, 0.1], None],
    ]

Let's use pandas DataFrame visualisation in order to display this table

In [ ]:
transition_probabilities_df = pd.DataFrame(transition_probabilities).rename_axis('Actions', axis=1)
transition_probabilities_df.index.name="State"
transition_probabilities_df

We also define the rewards, each actions couple state/action will lead to.

In [ ]:
rewards = [
        [[+10, 0, 0], [0, 0, 0], [0, 0, 0]],
        [[0, 0, 0], [0, 0, 0], [0, 0, -50]],
        [[0, 0, 0], [+40, 0, 0], [0, 0, 0]],
    ]

rewards_df = pd.DataFrame(rewards).rename_axis('Actions', axis=1)
rewards_df.index.name="State"
rewards_df

And the list of possible action that can be taken at each state.

In [ ]:
possible_actions = [[0, 1, 2], [0, 2], [1]]

Finally we define now a class that will act as a Gym environment. 

* The environement is the MDP.
* The observation is the current step.
* The action possible are the three actions we previously define.

In [ ]:
class MDPEnvironment(object):
    def __init__(self, start_state=0):
        self.start_state=start_state
        self.reset()
    def reset(self):
        self.total_rewards = 0
        self.state = self.start_state
    def step(self, action):
        next_state = np.random.choice(range(3), p=transition_probabilities[self.state][action])
        reward = rewards[self.state][action][next_state]
        self.state = next_state
        self.total_rewards += reward
        return self.state, reward

In [ ]:
def run_episode(policy, n_steps, start_state=0):
    env = MDPEnvironment()
    for step in range(n_steps):
        action = policy(env.state)
        state, reward = env.step(action)
    return env.total_rewards


### Hard Coded Policy

Let's first implement a random policy, as a baseline we want to improve

In [ ]:
def policy_random(state):
    return np.random.choice(possible_actions[state])


all_score = []
for episode in range(1000):
    all_score.append(run_episode(policy_random, n_steps=100))
print("Summary: mean={:.1f}, std={:1f}, min={}, max={}".format(np.mean(all_score), np.std(all_score), np.min(all_score), np.max(all_score)))


**Exercise** Which policy would be the safest? The more risky? Implement it and test it. What can you say about their results?

In [ ]:
# %load solutions/exercise_2_1.py

### Q-Learning

Let's know implement Q-learning algorithm to learn a better policy!

Q-Learning works by watching an agent play (e.g., randomly) and gradually improving its estimates of the Q-Values. 
Once it has accurate Q-Value estimates (or close enough), then the optimal policy consists in choosing the action that has the highest Q-Value (i.e., the greedy policy).

We first initiate:
* the different parameters (learning_rate $\alpha$ and the discount rate $\gamma$}
* The number of step to play
* The exploration policy (random one)
* The Q-values tables

In [ ]:
n_states = 3
n_actions = 3
n_steps = 20
alpha = 0.01  #<-- Learning Rate
gamma = 0.99  #<-- The discount rate


 
exploration_policy = policy_random #<-- Policy that we will play during exploration
q_values = np.full((n_states, n_actions), -np.inf) #<-- Policy that we will be updated
for state, actions in enumerate(possible_actions):
    q_values[state][actions]=0
q_values

**Exercise**
Run *n_steps* over the MDP and update the Q-values table at each step according to the Q-learning iteration algorithm

In [ ]:
env = MDPEnvironment()
for step in range(n_steps):    


In [ ]:
# %load solutions/exercise_2_21.py

**Exercise** How do we defined the optimal policy from the computed Q_values? Implement it.

In [ ]:
def optimal_policy(state):
    return 

In [ ]:
# %load solutions/exercise_2_22.py

Compute its performance.

In [ ]:
all_totals = []
for episode in range(1000):
    all_totals.append(run_episode(optimal_policy, n_steps=100))
print("Summary: mean={:.1f}, std={:1f}, min={}, max={}".format(np.mean(all_totals), np.std(all_totals), np.min(all_totals), np.max(all_totals)))
print()

**Q** We uses the Q learning iteration algorithm to learn the best policy. Would it have been possible to use a different algorithm here? Why?

## DEEP Q Learning on *Pacman-Like* Game

We have seen how to learn an optimal policy on a Markov Decision Process. 

Let's now try to learn how to play a Video Game Like Pacman. 

We won't use directly Pacman here (Even if it's available on Gym environment (see below)). The Pacman game requires GPU to be trained and may need some try and parameter optimization to easily converged. 

Hence, we will start with and simpler environment which looks like Pacman : the [gridworld](https://github.com/awjuliani/DeepRL-Agents/blob/master/gridworld.py) environment.




### Grid World environment

We have a 5x5 blocs where blue, green and red squares. In the environment the agent controls a blue square, and the goal is to navigate to the green squares (reward +1) while avoiding the red squares (reward -1).

#### Observation

The observation is the image itself which is a 84x84x3 images.

#### Actions

Num | Action
--- | ---
0 | Go Up
1 | Go Down
2 | Go Left
3 | Go Right


#### Reward
Reward is 1 for every green square taken and -1 when a red square is taken

#### End epsiode
There are no condition limit for an episode to finish

We initiate the environment

In [ ]:
from gridworld import gameEnv
env = gameEnv(partial=False, size=5)

**Exercise**: Let's play a game manually to understand how it works. Fill the actions_list with step in order to win points according to the intialised environment.

In [ ]:
frames = []
cum_reward=0
actions_list=[]
for step in actions_list:
    state, reward , end = env.step(step)
    frames.append(state)
    cum_reward+=reward
cum_reward
HTML(plot_animation(frames).to_html5_video())

### Experience Replay Buffer

To make DQNs work you need an **Experience Replay Buffer**. The basic idea is that by storing an agent’s experiences, and then randomly drawing batches of them to train the network, we can more robustly learn to perform well in the task. The Experience Replay buffer stores a fixed number of recent memories, and as new ones come in, old ones are removed. When the time comes to train, we simply draw a uniform batch of random memories from the buffer, and train our network with them. For our DQN, we will build a simple class that handles storing and retrieving memories.

In [ ]:
class ExperienceReplay:
    def __init__(self, buffer_size=50000):
        """ Data structure used to hold game experiences """
        # Buffer will contain [state,action,reward,next_state,done]
        self.buffer = []
        self.buffer_size = buffer_size

    def add(self, experience):
        """ Adds list of experiences to the buffer """
        # Extend the stored experiences
        self.buffer.extend(experience)
        # Keep the last buffer_size number of experiences
        self.buffer = self.buffer[-self.buffer_size:]

    def sample(self, size):
        """ Returns a sample of experiences from the buffer """
        sample_idxs = np.random.randint(len(self.buffer), size=size)
        sample_output = [self.buffer[idx] for idx in sample_idxs]
        sample_output = np.reshape(sample_output, (size, -1))
        return sample_output

### Architecture and loss of network


#### Q-Learning
Here are the Q-learning formula used above

$$target = R(s,a,s')+\gamma \max\limits_{a'}Q_k(s',a')$$
$$Q_{k+1}(s,a)\leftarrow(1-a)Q_k(s,a)+\alpha[target]$$

if the combinations of states and actions are too large, the memory and the computation requirement for Q will be too high.

#### Deep Q-learning

To address that, we switch to a deep network Q (DQN) to approximate Q(s, a). The learning algorithm is called Deep Q-learning. With the new approach, we generalize the approximation of the Q-value function rather than remembering the solutions.

$$target = R(s,a,s')+\gamma \max\limits_{a'}Q_k(s',a')$$
$$\theta_{k+1} \leftarrow \theta_k - \alpha\nabla_{\theta}\mathbb{E}_{s\sim'P(s'|s,a)} [(Q_{\theta}(s,a)-target(s'))^2]_{\theta=\theta_k} $$

We define below a convolutional network. 

**Exercise** Complete the code of the convolutional network below with the input and output shape in order to make it work with our problem.

In [ ]:
class Qnetwork():
    def __init__(self):
        self.inputs = kl.Input(shape=#COMPLETE, name="main_input")

        self.model = kl.Conv2D(
            filters=32,
            kernel_size=[8, 8],
            strides=[4, 4],
            activation="relu",
            padding="valid",
            name="conv1")(self.inputs)
        self.model = kl.Conv2D(
            filters=64,
            kernel_size=[4, 4],
            strides=[2, 2],
            activation="relu",
            padding="valid",
            name="conv2")(self.model)
        self.model = kl.Conv2D(
            filters=64,
            kernel_size=[3, 3],
            strides=[1, 1],
            activation="relu",
            padding="valid",
            name="conv3")(self.model)
        self.model = kl.Conv2D(
            filters=512,
            kernel_size=[7, 7],
            strides=[1, 1],
            activation="relu",
            padding="valid",
            name="conv4")(self.model)

        self.model = kl.Flatten()(self.model)
        self.model = kl.Dense(256, activation="relu")(self.model)
        self.model = kl.Dense(#COMPLETE, activation = "linear")(self.model)
        self.model = km.Model(self.inputs, self.model)
        self.model.compile("adam", "mse")
        self.model.optimizer.lr = 0.0001

In [ ]:
# %load solutions/exercise_2_3.py

Init a model in order to display it's summary.

In [ ]:
main_qn = Qnetwork()
main_qn.model.summary()

### DQN Class

#### Separated Target network
Here is how the target is supposed to be computed in order to train or Deep-Q network.

$$target = R(s,a,s')+\gamma \max\limits_{a'}Q_k(s',a';\theta) $$


We build a deep network to learn the values of Q but its target values are changing as we know things better. As shown below, the target values for Q depends on Q itself, we are chasing a non-stationary target. So we use a different network to generate the target, that will be updated from time to time


$$target = R(s,a,s')+\gamma \max\limits_{a'}Q_k(s',a';\theta_{target}) $$


**Exercise**  Complete the class below to train a DQN network

1. Pay attention to all the parameters define in the init and make sure you understand its.
2. Look at the train function that allow to run the pseudo code described below.
3. Go inside all other function and complete the code where you'll find a TODO mark.


#### Pseudo Code
* while num_episode < self.num_episodes:

    1. Run one complete episode:
        * If less than *pre_train_episode* has been played:
            * Play randomly
        * Else
            * play randomly with probability *prob_random* (Exploration) else play with the *main_q* network (Exploitation).
        * Store the episode in the *ReplayBuffer*
    2. If more than *pre_train_episode* has been played
                * Decrease the probability of playing randomly (in order to use more Exploitation than Exploration (e-greedy policy).
                * Every 5 episodes played:
                    * Train the episode using episode from the *ReplayBuffer* and using the target build from the *target_q network*
                    * Update the *target_q* weight with *main_q* weight.

In [ ]:
class DQN:
    def __init__(self):
        self.batch_size = 64  # How many experiences to use for each training step
        self.num_epochs = 20  # How many epochs to train
        self.update_freq = 5  # How often you update the network
        self.y = 0.99  # Discount factor
        self.prob_random_start = 0.6  # Starting chance of random action
        self.prob_random_end = 0.1  # Ending chance of random action
        self.annealing_steps = 1000.  # Steps of training to reduce from start_e -> end_e
        self.num_episodes = 10000  # How many episodes of game environment to train
        self.pre_train_episodes = 100  # Number of episodes of random actions
        self.max_num_step = 50  # Maximum allowed episode length
        self. goal = 15

        # Reset everything
        K.clear_session()

        #TODO: Setup both the main and the target network.
        

        #TODO Setup our experience replay
        self.experience_replay = ExperienceReplay()

    def update_target_graph(self):
        # TODO : This function will be called every  self.update_freq train step, in order to copy all the weight of the main Q network to the target Q network


    def run_one_episode(self, num_episode, prob_random):
        # Create an experience replay for the current episode
        episode_buffer = ExperienceReplay()

        # Get the game state from the environment
        state = env.reset()

        done = False  # Game is complete
        sum_rewards = 0  # Running sum of rewards in episode
        cur_step = 0  # Running sum of number of steps taken in episode

        while cur_step < self.max_num_step and not done:
            cur_step += 1
            
            # TODO while that we haven't reach the  self.pre_train_episodes number of pre train episod, the action for the given state is choosen randomly. 
            # WHen self.pre_train_episode is reached, choose randomly tha ction with a 'prob_random' probability else use the main Q network
            

            # TODO Use action and retrieve the next state, reward and done

            # Setup the episode to be stored in the episode buffer
            episode = np.array([[state], action, reward, [next_state], done])
            episode = episode.reshape(1, -1)

            # Store the experience in the episode buffer
            episode_buffer.add(episode)

            # Update the running rewards
            sum_rewards += reward

            # Update the state
            state = next_state

        return episode_buffer, sum_rewards, cur_step

    def train_one_step(self):
        # TODO  Sample a batch from the self.experience_replay class
        train_batch = self.experience_replay.sample(self.batch_size)

        # Separate the batch into its components
        train_state, train_action, train_reward, \
        train_next_state, train_done = train_batch.T

        # Convert the action array into an array of ints so they can be used for indexing
        train_action = train_action.astype(np.int)

        # Stack the train_state and train_next_state for learning
        train_state = np.vstack(train_state)
        train_next_state = np.vstack(train_next_state)

        #TODO Build the target in order to use them to build our network


        # Train the main model
        loss = self.main_qn.model.train_on_batch(train_state, target_q)
        return loss


    def train(self):

        # Make the networks equal
        self.update_target_graph()

        # We'll begin by acting complete randomly. As we gain experience and improve,
        # we will begin reducing the probability of acting randomly, and instead
        # take the actions that our Q network suggests
        prob_random = self.prob_random_start
        prob_random_drop = (self.prob_random_start - self.prob_random_end) / self.annealing_steps

        num_steps = []  # Tracks number of steps per episode
        rewards = []  # Tracks rewards per episode
        print_every = 50  # How often to print status
        losses = [0]  # Tracking training losses
        num_episode = 0

        while num_episode < self.num_episodes:

            episode_buffer, sum_rewards, cur_step = self.run_one_episode(num_episode, prob_random)

            if num_episode > self.pre_train_episodes:
                # Training the network

                if prob_random > self.prob_random_end:
                    # Drop the probability of a random action
                    prob_random -= prob_random_drop

                if num_episode % self.update_freq == 0:
                    for num_epoch in range(self.num_epochs):
                        loss = self.train_one_step()
                        losses.append(loss)

                    # Update the target model with values from the main model
                    self.update_target_graph()

            # Increment the episode
            num_episode += 1

            self.experience_replay.add(episode_buffer.buffer)
            num_steps.append(cur_step)
            rewards.append(sum_rewards)

            if num_episode % print_every == 0:
                # Print progress
                mean_loss = np.mean(losses[-(print_every * self.num_epochs):])

                print("Num episode: {} Mean reward: {:0.4f} Prob random: {:0.4f}, Loss: {:0.04f}".format(
                    num_episode, np.mean(rewards[-print_every:]), prob_random, mean_loss))
                if np.mean(rewards[-print_every:]) >= self.goal:
                    print("Training complete!")
                    break



In [ ]:
# %load solutions/exercise_2_4.py

Train the model

In [ ]:
dqn = DQN()
dqn.train()

**Exercise** Play a pacman game and display the result of a game in video.

In [ ]:
# %load solutions/exercise_2_5.py

### DQN Improvement

DQN has  been improved recently by using two different tricks (among others):

* Dueling
* Double DQN

Implement these two solutions.

#### Dueling

See the dueling architecture : 


In order to explain the reasoning behind the architecture changes that Dueling DQN makes, we need to first explain some a few additional reinforcement learning terms. The Q-values that we have been discussing so far correspond to how good it is to take a certain action given a certain state. This can be written as Q(s,a). This action given state can actually be decomposed into two more fundamental notions of value. The first is the value function V(s), which says simple how good it is to be in any given state. The second is the advantage function A(a), which tells how much better taking a certain action would be compared to the others. We can then think of Q as being the combination of V and A. More formally:
$$Q(s,a) =V(s) + A(a)$$
The goal of Dueling DQN is to have a network that separately computes the advantage and value functions, and combines them back into a single Q-function only at the final layer. It may seem somewhat pointless to do this at first glance. Why decompose a function that we will just put back together? The key to realizing the benefit is to appreciate that our reinforcement learning agent may not need to care about both value and advantage at any given time. For example: imagine sitting outside in a park watching the sunset. It is beautiful, and highly rewarding to be sitting there. No action needs to be taken, and it doesn’t really make sense to think of the value of sitting there as being conditioned on anything beyond the environmental state you are in. We can achieve more robust estimates of state value by decoupling it from the necessity of being attached to specific actions.

In [ ]:
# %load solutions/exercise_2_6.py

In [ ]:
main_qn = Qnetwork()
main_qn.model.summary()

#### Double DQN


This lead to often overestimates the Q-values of the potential actions to take in a given state. While this would be fine if all actions were always overestimates equally, there was reason to believe this wasn’t the case. You can easily imagine that if certain suboptimal actions regularly were given higher Q-values than optimal actions, the agent would have a hard time ever learning the ideal policy. In order to correct for this, the authors of DDQN paper propose a simple trick: instead of taking the max over Q-values when computing the target-Q value for our training step, we use our primary network to chose an action, and our target network to generate the target Q-value for that action. By decoupling the action choice from the target Q-value generation, we are able to substantially reduce the overestimation, and train faster and more reliably. Below is the new DDQN equation for updating the target value.


$$target = R(s,a,s')+\gamma Q_k(s',argmax_aQ(s',a;\theta);\theta_{target}) $$



In [ ]:
# %load solutions/exercise_2_7.py

In [ ]:
dqn = DQN()
dqn.train()

## Real Pacman!

### Creating the MsPacman environment

In [ ]:
env = gym.make("MsPacman-v0")
obs = env.reset()
obs.shape

In [ ]:
env.action_space

### Preprocessing

Preprocessing the images is optional but greatly speeds up training.

In [ ]:
mspacman_color = 210 + 164 + 74

def preprocess_observation(obs):
    img = obs[1:176:2, ::2] # crop and downsize
    img = img.sum(axis=2) # to greyscale
    img[img==mspacman_color] = 0 # Improve contrast
    img = (img // 3 - 128).astype(np.int8) # normalize from -128 to 127
    return img.reshape(88, 80, 1)/128

img = preprocess_observation(obs)
img.shape

Note: the `preprocess_observation()` function is slightly different from the one in the book: instead of representing pixels as 64-bit floats from -1.0 to 1.0, it represents them as signed bytes (from -128 to 127). The benefit is that the replay memory will take up roughly 8 times less RAM (about 6.5 GB instead of 52 GB). The reduced precision has no visible impact on training.

In [ ]:
plt.figure(figsize=(11, 7))
plt.subplot(121)
plt.title("Original observation (160×210 RGB)")
plt.imshow(obs)
plt.axis("off")
plt.subplot(122)
plt.title("Preprocessed observation (88×80 greyscale)")
plt.imshow(img.reshape(88, 80), interpolation="nearest", cmap="gray")
plt.axis("off")
plt.show()